In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random as rnd

In [32]:
Original_dataset = pd.read_csv('train.csv')

In [33]:
test_dataset = pd.read_csv('test.csv')

In [34]:
Original_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
Original_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [36]:
Original_dataset.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [37]:
Original_dataset.describe(include = ['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Abbott, Mr. Rossmore Edward",male,CA. 2343,B96 B98,S
freq,1,577,7,4,644


In [38]:
Original_dataset[['Pclass','Survived']].groupby(['Pclass'],as_index = False).mean().sort_values(by = 'Survived',ascending = False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [39]:
train = Original_dataset.drop(columns=["Ticket","Cabin"])
test = test_dataset.drop(columns=["Ticket","Cabin"])
Combine = [train,test]

In [40]:
for dataset in Combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.',expand = False)
    
pd.crosstab(train['Title'],train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [41]:
for dataset in Combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [42]:
train = train.drop(columns="Name")
test = test.drop(columns="Name")
combine = [train,test]

In [43]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].fillna(0)

In [44]:
for dataset in combine:
    dataset["Sex"] = dataset["Sex"].map({'female':1,'male':0}).astype(int)

In [45]:
guess_ages = np.zeros((2,3))
guess_ages

array([[0., 0., 0.],
       [0., 0., 0.]])

In [46]:
import random as rnd
for dataset in combine:
    for i in range(0,2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            age_mean = guess_df.mean()
            age_std = guess_df.std()
            age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            #age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,0,22,1,0,7.2500,S,Mr
1,2,1,1,1,38,1,0,71.2833,C,Mrs
2,3,1,3,1,26,0,0,7.9250,S,Miss
3,4,1,1,1,35,1,0,53.1000,S,Mrs
4,5,0,3,0,35,0,0,8.0500,S,Mr
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27,0,0,13.0000,S,Rare
887,888,1,1,1,19,0,0,30.0000,S,Miss
888,889,0,3,1,28,1,2,23.4500,S,Miss
889,890,1,1,0,26,0,0,30.0000,C,Mr


In [47]:
freq_embarked = train.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset["Emmbarked"] = dataset["Embarked"].fillna(freq_embarked) 

In [48]:
train = pd.get_dummies(train, columns=['Embarked','Title'],drop_first = True)
test = pd.get_dummies(test, columns=['Embarked','Title'],drop_first = True)

In [49]:
test['Fare'].fillna(test['Fare'].dropna().median(),inplace = True)

In [60]:
train = train.drop(columns = "Emmbarked")
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,1,0,3,0,22,1,0,7.2500,0,1,0,1,0,0
1,2,1,1,1,38,1,0,71.2833,0,0,0,0,1,0
2,3,1,3,1,26,0,0,7.9250,0,1,1,0,0,0
3,4,1,1,1,35,1,0,53.1000,0,1,0,0,1,0
4,5,0,3,0,35,0,0,8.0500,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27,0,0,13.0000,0,1,0,0,0,1
887,888,1,1,1,19,0,0,30.0000,0,1,1,0,0,0
888,889,0,3,1,28,1,2,23.4500,0,1,1,0,0,0
889,890,1,1,0,26,0,0,30.0000,0,0,0,1,0,0


In [62]:
from sklearn.preprocessing import StandardScaler ,Normalizer
scaled_features = StandardScaler().fit_transform(train.values)
scaled_features_df = pd.DataFrame(scaled_features, index=train.index, columns=train.columns)
scaled_features_t = StandardScaler().fit_transform(test.values)
scaled_features_tdf = pd.DataFrame(scaled_features_t, index=test.index, columns=test.columns)
scaled_features_df

ValueError: could not convert string to float: 'Q'